In [9]:
HOME = '/data/git/shopee-product-matching'
pdata = f'{HOME}/data/shopee-product-matching'
!ls {pdata}/train_images/  | head

0000a68812bc7e98c42888dfb1c07da0.jpg
00039780dfc94d01db8676fe789ecd05.jpg
000a190fdd715a2a36faed16e2c65df7.jpg
00117e4fc239b1b641ff08340b429633.jpg
00136d1cf4edede0203f32f05f660588.jpg
0013e7355ffc5ff8fb1ccad3e42d92fe.jpg
00144a49c56599d45354a1c28104c039.jpg
0014f61389cbaa687a58e38a97b6383d.jpg
0019a3c6755a194cb2e2c12bfc63972e.jpg
001be52b2beec40ddc1d2d7fc7a68f08.jpg
ls: write error: Broken pipe


In [11]:
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b0')

Loaded pretrained weights for efficientnet-b0


In [12]:
from PIL import Image
from torchvision import transforms
# https://github.com/lukemelas/EfficientNet-PyTorch
tfms = transforms.Compose([transforms.Resize(224), transforms.ToTensor(),
transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),])
img = tfms(Image.open(f'{pdata}/train_images/8115b171a6569912d0ec6a9e9a39d62a.jpg')).unsqueeze(0)
print(img.shape) # torch.Size([1, 3, 224, 224])
features = model.extract_features(img)
print(features.shape) # torch.Size([1, 1280, 7, 7])


torch.Size([1, 3, 224, 224])
torch.Size([1, 1280, 7, 7])


In [17]:
from transformers import AutoModel, AutoTokenizer
import torch

In [53]:
# 'bert-base-multilingual-cased',
# 'xlm-roberta-base',
# 'pvl/labse_bert'
nm_mdl = 'pvl/labse_bert'
model = AutoModel.from_pretrained(nm_mdl)
tokenizer = AutoTokenizer.from_pretrained(nm_mdl, do_lower_case=False)

In [54]:
def mk_tensors(txt, tokenizer, maxlen):
    tok_res = tokenizer(
        txt, truncation=True, pad_to_max_length=True, max_length=maxlen
    )
    input_ids = tok_res["input_ids"]
    token_type_ids = tok_res["token_type_ids"]
    attention_mask = tok_res["attention_mask"]
    input_ids = torch.tensor(input_ids)
    token_type_ids = torch.tensor(token_type_ids)
    attention_mask = torch.tensor(attention_mask)
    return input_ids, attention_mask, token_type_ids

In [64]:
input_ids, attention_mask, token_type_ids = mk_tensors(["abc dadf adsf"], tokenizer, 32)

In [65]:
input_ids, attention_mask, token_type_ids

(tensor([[   101, 163701,  26808,  15625,  45328,  15625,    102,      0,      0,
               0,      0,      0,      0,      0,      0,      0,      0,      0,
               0,      0,      0,      0,      0,      0,      0,      0,      0,
               0,      0,      0,      0,      0]]),
 tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0]]),
 tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0]]))

In [79]:
type(model)

transformers.models.bert.modeling_bert.BertModel

https://huggingface.co/transformers/model_doc/bert.html#bertmodel

    last_hidden_state (torch.FloatTensor of shape (batch_size, sequence_length, hidden_size)) – Sequence of hidden-states at the output of the last layer of the model.

    pooler_output (torch.FloatTensor of shape (batch_size, hidden_size)) – Last layer hidden-state of the first token of the sequence (classification token) further processed by a Linear layer and a Tanh activation function. The Linear layer weights are trained from the next sentence prediction (classification) objective during pretraining.

In [87]:
output = model(input_ids, attention_mask, token_type_ids)

In [88]:
len(output), output.keys()

(2, odict_keys(['last_hidden_state', 'pooler_output']))

In [89]:
for k,v in output.items():
    print(k, v.shape)

last_hidden_state torch.Size([1, 32, 768])
pooler_output torch.Size([1, 768])


https://github.com/huggingface/transformers/issues/1328#issuecomment-534956703

    Both would probably work, but I agree that streamlining is a good idea. In their paper, BERT gets the best results by concatenating the last four layers, so what I always use is something like this (from the top of my head):

    outputs = self.bert(input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        position_ids=position_ids, 
                        head_mask=head_mask)

    hidden_states = outputs[1]
    pooled_output = torch.cat(tuple([hidden_states[i] for i in [-4, -3, -2, -1]]), dim=-1)
    pooled_output = pooled_output[:, 0, :]
    pooled_output = self.dropout(pooled_output)
    # classifier of course has to be 4 * hidden_dim, because we concat 4 layers
    logits = self.classifier(pooled_output)

In [90]:
output = model(input_ids, attention_mask, token_type_ids, output_hidden_states=True)

In [91]:
len(output)

3

In [92]:
output.keys()

odict_keys(['last_hidden_state', 'pooler_output', 'hidden_states'])

In [94]:
len(output.hidden_states)

13

In [95]:
[o.shape for o in output.hidden_states]

[torch.Size([1, 32, 768]),
 torch.Size([1, 32, 768]),
 torch.Size([1, 32, 768]),
 torch.Size([1, 32, 768]),
 torch.Size([1, 32, 768]),
 torch.Size([1, 32, 768]),
 torch.Size([1, 32, 768]),
 torch.Size([1, 32, 768]),
 torch.Size([1, 32, 768]),
 torch.Size([1, 32, 768]),
 torch.Size([1, 32, 768]),
 torch.Size([1, 32, 768]),
 torch.Size([1, 32, 768])]